## एम्बेडिंग्ज

आपल्या मागील उदाहरणात, आम्ही `vocab_size` लांबीच्या उच्च-आयामी बॅग-ऑफ-वर्ड्स व्हेक्टरवर कार्य केले आणि कमी-आयामी स्थानिक प्रतिनिधित्व व्हेक्टर स्पष्टपणे विरळ वन-हॉट प्रतिनिधित्वात रूपांतरित केले. हे वन-हॉट प्रतिनिधित्व मेमरीसाठी कार्यक्षम नाही. याशिवाय, प्रत्येक शब्द स्वतंत्रपणे विचारात घेतला जातो, त्यामुळे वन-हॉट एन्कोड केलेले व्हेक्टर शब्दांमधील अर्थपूर्ण समानता व्यक्त करत नाहीत.

या युनिटमध्ये, आपण **News AG** डेटासेटचा अभ्यास सुरू ठेवू. सुरुवात करण्यासाठी, डेटा लोड करूया आणि मागील युनिटमधील काही संकल्पना परिभाषित करूया.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### एम्बेडिंग म्हणजे काय?

**एम्बेडिंग** ही कल्पना म्हणजे शब्दांचे प्रतिनिधित्व कमी-आयामी घन गटांद्वारे करणे, जे त्या शब्दाचा अर्थपूर्ण अर्थ प्रतिबिंबित करतात. आपण नंतर कसे अर्थपूर्ण शब्द एम्बेडिंग तयार करायचे यावर चर्चा करू, पण सध्या एम्बेडिंगला शब्द वेक्टरची आयामसंख्या कमी करण्याचा एक मार्ग म्हणून विचार करूया.

तर, एक एम्बेडिंग लेयर शब्दाला इनपुट म्हणून घेते आणि निर्दिष्ट `embedding_size` च्या आउटपुट वेक्टरमध्ये रूपांतरित करते. एका अर्थाने, हे `Dense` लेयरसारखेच आहे, पण एक-हॉट एन्कोडेड वेक्टर इनपुट म्हणून घेण्याऐवजी, हे शब्द क्रमांक घेऊ शकते.

आपल्या नेटवर्कमध्ये पहिल्या लेयर म्हणून एम्बेडिंग लेयर वापरल्याने, आपण बॅग-ऑफ-वर्ड्स मॉडेलऐवजी **एम्बेडिंग बॅग** मॉडेलकडे स्विच करू शकतो, जिथे आपण प्रथम आपल्या मजकुरातील प्रत्येक शब्द त्याच्या संबंधित एम्बेडिंगमध्ये रूपांतरित करतो, आणि नंतर त्या सर्व एम्बेडिंग्सवर काही एकत्रित फंक्शन (जसे की `sum`, `average` किंवा `max`) गणना करतो.

![पाच अनुक्रम शब्दांसाठी एम्बेडिंग वर्गीकरण दाखवणारी प्रतिमा.](../../../../../translated_images/mr/embedding-classifier-example.b77f021a7ee67eee.webp)

आपल्या वर्गीकरण न्यूरल नेटवर्कमध्ये खालील लेयर्स असतात:

* `TextVectorization` लेयर, जी स्ट्रिंगला इनपुट म्हणून घेते आणि टोकन क्रमांकांचा टेन्सर तयार करते. आपण काही वाजवी शब्दसंग्रह आकार `vocab_size` निर्दिष्ट करू, आणि कमी-वारंवार वापरलेले शब्द दुर्लक्षित करू. इनपुट आकार 1 असेल, आणि आउटपुट आकार $n$ असेल, कारण आपल्याला $n$ टोकन्स मिळतील, ज्यामध्ये प्रत्येक टोकन 0 ते `vocab_size` मधील क्रमांक असतील.
* `Embedding` लेयर, जी $n$ क्रमांक घेते आणि प्रत्येक क्रमांकाला दिलेल्या लांबीच्या घन गटात (उदा. आपल्या उदाहरणात 100) कमी करते. त्यामुळे, $n$ आकाराचा इनपुट टेन्सर $n\times 100$ आकाराच्या टेन्सरमध्ये रूपांतरित होईल.
* एकत्रीकरण लेयर, जी या टेन्सरचा पहिल्या अक्षावर सरासरी काढते, म्हणजेच ती वेगवेगळ्या शब्दांसाठी असलेल्या सर्व $n$ इनपुट टेन्सर्सची सरासरी काढेल. हे लेयर अंमलात आणण्यासाठी, आपण `Lambda` लेयर वापरू, आणि त्यात सरासरी काढण्याचे फंक्शन पास करू. आउटपुटचा आकार 100 असेल, आणि तो संपूर्ण इनपुट अनुक्रमाचे संख्यात्मक प्रतिनिधित्व असेल.
* अंतिम `Dense` रेषीय वर्गीकरणकर्ता.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` च्या प्रिंटआउटमध्ये, **output shape** कॉलममध्ये, पहिला टेन्सर डायमेंशन `None` मिनीबॅचच्या आकाराशी संबंधित आहे, आणि दुसरा टोकन सिक्वेन्सच्या लांबीशी संबंधित आहे. मिनीबॅचमधील सर्व टोकन सिक्वेन्सची लांबी वेगवेगळी असते. यावर कसे काम करायचे, याबद्दल आपण पुढील विभागात चर्चा करू.

आता नेटवर्क प्रशिक्षण देऊया:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **टीप** की आम्ही डेटाच्या उपसमुच्च्यावर आधारित व्हेक्टरायझर तयार करत आहोत. प्रक्रिया वेगवान करण्यासाठी हे केले जाते, आणि यामुळे अशी परिस्थिती निर्माण होऊ शकते की आमच्या मजकुरातील सर्व टोकन्स शब्दसंग्रहात उपस्थित नसतील. अशा परिस्थितीत, त्या टोकन्स दुर्लक्षित केले जातील, ज्यामुळे अचूकतेत थोडीशी घट होऊ शकते. तथापि, वास्तविक जीवनात मजकुराचा उपसमुच्चा अनेकदा चांगल्या शब्दसंग्रहाचा अंदाज देतो.


### चल बदलत्या अनुक्रमणिकेच्या आकारांशी व्यवहार करणे

चला समजून घेऊया की मिनीबॅचेसमध्ये प्रशिक्षण कसे होते. वरील उदाहरणात, इनपुट टेन्सरचे परिमाण 1 आहे, आणि आपण 128-लांब मिनीबॅचेस वापरतो, त्यामुळे टेन्सरचा वास्तविक आकार $128 \times 1$ आहे. मात्र, प्रत्येक वाक्यातील टोकन्सची संख्या वेगळी असते. जर आपण `TextVectorization` स्तर एका इनपुटवर लागू केला, तर परत मिळणाऱ्या टोकन्सची संख्या वेगळी असते, कारण ती मजकूर कसा टोकनाइझ केला जातो यावर अवलंबून असते:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


तथापि, जेव्हा आपण वेक्टरायझर अनेक अनुक्रमांवर लागू करतो, तेव्हा त्याला आयताकृती आकाराचा टेन्सर तयार करावा लागतो, त्यामुळे तो न वापरलेल्या घटकांना PAD टोकनने (जो आपल्या बाबतीत शून्य आहे) भरतो:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

येथे आपण एम्बेडिंग्स पाहू शकतो:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

## सिमॅंटिक एम्बेडिंग्स: Word2Vec

आपल्या मागील उदाहरणात, एम्बेडिंग लेयरने शब्दांना व्हेक्टर प्रतिनिधित्वात मॅप करणे शिकले, परंतु या प्रतिनिधित्वांना सिमॅंटिक अर्थ नव्हता. असे व्हेक्टर प्रतिनिधित्व शिकणे चांगले होईल ज्यामध्ये समान शब्द किंवा समानार्थी शब्द काही व्हेक्टर अंतराच्या दृष्टीने (उदाहरणार्थ, युक्लिडियन अंतर) एकमेकांच्या जवळ असतील.

हे साध्य करण्यासाठी, आपल्याला [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) सारख्या तंत्राचा वापर करून मोठ्या प्रमाणातील मजकूरावर आपला एम्बेडिंग मॉडेल प्रीट्रेन करणे आवश्यक आहे. हे दोन मुख्य आर्किटेक्चरवर आधारित आहे जे शब्दांचे वितरित प्रतिनिधित्व तयार करण्यासाठी वापरले जातात:

 - **कंटिन्युअस बॅग-ऑफ-वर्ड्स** (CBoW), ज्यामध्ये आपण मॉडेलला आजूबाजूच्या संदर्भातून शब्दाचा अंदाज लावण्यासाठी प्रशिक्षित करतो. दिलेल्या ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ मध्ये, मॉडेलचे उद्दिष्ट $(W_{-2},W_{-1},W_1,W_2)$ पासून $W_0$ अंदाज लावणे आहे.
 - **कंटिन्युअस स्किप-ग्राम** CBoW च्या उलट आहे. मॉडेल सध्याचा शब्द अंदाज लावण्यासाठी संदर्भ शब्दांच्या विंडोचा वापर करते.

CBoW जलद आहे, आणि स्किप-ग्राम जरी हळू असला तरी, तो दुर्मिळ शब्दांचे प्रतिनिधित्व अधिक चांगल्या प्रकारे करतो.

![CBoW आणि स्किप-ग्राम अल्गोरिदम्स शब्दांना व्हेक्टरमध्ये रूपांतरित करण्यासाठी दाखवणारी प्रतिमा.](../../../../../translated_images/mr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News डेटासेटवर प्रीट्रेन केलेल्या Word2Vec एम्बेडिंगसह प्रयोग करण्यासाठी, आपण **gensim** लायब्ररीचा वापर करू शकतो. खाली 'neural' शब्दाशी सर्वाधिक समान असलेले शब्द शोधले आहेत.

> **टीप:** जेव्हा तुम्ही प्रथम शब्द व्हेक्टर तयार करता, तेव्हा त्यांना डाउनलोड करणे थोडा वेळ घेऊ शकते!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


आम्ही शब्दातून व्हेक्टर एम्बेडिंग देखील काढू शकतो, जे वर्गीकरण मॉडेल प्रशिक्षणासाठी वापरले जाऊ शकते. एम्बेडिंगमध्ये 300 घटक आहेत, परंतु स्पष्टतेसाठी येथे आम्ही फक्त व्हेक्टरचे पहिले 20 घटक दाखवतो:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

सामान्य अर्थाच्या एम्बेडिंग्सबद्दलची महान गोष्ट म्हणजे तुम्ही अर्थाच्या आधारावर व्हेक्टर एन्कोडिंगमध्ये फेरफार करू शकता. उदाहरणार्थ, आपण असे विचारू शकतो की *राजा* आणि *स्त्री* या शब्दांच्या जितके जवळजवळ व्हेक्टर प्रतिनिधित्व असेल आणि *पुरुष* या शब्दापासून जितके दूर असेल असा शब्द शोधा:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

वरील उदाहरणामध्ये काही अंतर्गत GenSym जादूचा वापर केला आहे, परंतु मूळ तर्कशास्त्र खूपच सोपे आहे. एम्बेडिंग्सबद्दल एक मनोरंजक गोष्ट म्हणजे तुम्ही एम्बेडिंग वेक्टरवर सामान्य वेक्टर ऑपरेशन्स करू शकता, आणि ते शब्दांच्या **अर्थांवर** होणाऱ्या ऑपरेशन्सचे प्रतिबिंबित करेल. वरील उदाहरण वेक्टर ऑपरेशन्सच्या दृष्टीने व्यक्त केले जाऊ शकते: आम्ही **KING-MAN+WOMAN** शी संबंधित वेक्टरची गणना करतो (`+` आणि `-` ऑपरेशन्स संबंधित शब्दांच्या वेक्टर प्रतिनिधींवर केले जातात), आणि नंतर त्या वेक्टरच्या जवळचा शब्द शब्दकोशात शोधतो:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **टीप**: आम्हाला *man* आणि *woman* वेक्टरमध्ये छोटे coefficients जोडावे लागले - ते काढून टाकून काय होते ते पाहा.

सर्वात जवळचा वेक्टर शोधण्यासाठी, आम्ही TensorFlow चा वापर करून आमच्या वेक्टर आणि शब्दसंग्रहातील सर्व वेक्टरमधील अंतराचा वेक्टर मोजतो, आणि नंतर `argmin` वापरून किमान शब्दाचा निर्देशांक शोधतो.


जरी Word2Vec शब्दांच्या अर्थपूर्णतेसाठी एक उत्कृष्ट पद्धत वाटत असली तरी, त्याला अनेक मर्यादा आहेत, जसे की खालीलप्रमाणे:

* CBoW आणि skip-gram मॉडेल्स हे **predictive embeddings** आहेत, आणि ते फक्त स्थानिक संदर्भाचा विचार करतात. Word2Vec जागतिक संदर्भाचा फायदा घेत नाही.
* Word2Vec शब्दांच्या **रूपविज्ञानाचा** विचार करत नाही, म्हणजेच शब्दाचा अर्थ त्याच्या वेगवेगळ्या भागांवर (जसे की मूळ शब्द) अवलंबून असतो.

**FastText** दुसऱ्या मर्यादेवर मात करण्याचा प्रयत्न करते आणि Word2Vec वर आधारित आहे. हे प्रत्येक शब्दासाठी आणि त्या शब्दामध्ये आढळणाऱ्या अक्षर n-grams साठी व्हेक्टर प्रतिनिधित्व शिकते. या प्रतिनिधित्वांच्या मूल्यांना प्रत्येक प्रशिक्षण टप्प्यावर एका व्हेक्टरमध्ये सरासरी केली जाते. जरी यामुळे पूर्व-प्रशिक्षणासाठी अतिरिक्त गणना लागते, तरीही यामुळे शब्दांच्या embeddings मध्ये उप-शब्द माहिती समाविष्ट होऊ शकते.

दुसरी पद्धत, **GloVe**, शब्दांच्या embeddings साठी वेगळा दृष्टिकोन वापरते, जो शब्द-संदर्भ मॅट्रिक्सच्या घटकांवर आधारित आहे. प्रथम, ती एक मोठी मॅट्रिक्स तयार करते जी वेगवेगळ्या संदर्भांमध्ये शब्दांच्या उपस्थितीची संख्या मोजते, आणि नंतर ती या मॅट्रिक्सचे कमी परिमाणांमध्ये प्रतिनिधित्व करण्याचा प्रयत्न करते, ज्यामुळे पुनर्रचना तोटा (reconstruction loss) कमी होतो.

gensim लायब्ररी या शब्दांच्या embeddings ला समर्थन देते, आणि तुम्ही वरील मॉडेल लोडिंग कोड बदलून त्यांच्यासोबत प्रयोग करू शकता.


## केरस मध्ये पूर्वप्रशिक्षित एम्बेडिंग्सचा वापर

आपण वरील उदाहरण बदलून आपल्या एम्बेडिंग लेयरमध्ये Word2Vec सारख्या सिमॅंटिक एम्बेडिंग्ससह मॅट्रिक्स पूर्वप्रस्थापित करू शकतो. पूर्वप्रशिक्षित एम्बेडिंग आणि मजकूर कॉर्पसची शब्दसंग्रह (vocabularies) बहुधा जुळणार नाहीत, त्यामुळे आपल्याला एक निवड करावी लागेल. येथे आपण दोन शक्य पर्यायांचा अभ्यास करतो: टोकनायझर शब्दसंग्रहाचा वापर करणे आणि Word2Vec एम्बेडिंग्समधील शब्दसंग्रहाचा वापर करणे.

### टोकनायझर शब्दसंग्रहाचा वापर

टोकनायझर शब्दसंग्रहाचा वापर करताना, शब्दसंग्रहातील काही शब्दांना Word2Vec एम्बेडिंग्स असतील, तर काही गहाळ असतील. गृहीत धरले की आपला शब्दसंग्रह आकार `vocab_size` आहे, आणि Word2Vec एम्बेडिंग व्हेक्टरची लांबी `embed_size` आहे, तर एम्बेडिंग लेयर `vocab_size`$\times$`embed_size` आकाराच्या वेट मॅट्रिक्सने दर्शविला जाईल. आपण हा मॅट्रिक्स शब्दसंग्रहामधून जाऊन भरू:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


ज्या शब्दांसाठी Word2Vec शब्दसंग्रहात शब्द उपलब्ध नाहीत, त्यांना शून्य म्हणून ठेवू शकतो किंवा एक यादृच्छिक वेक्टर तयार करू शकतो.

आता आपण प्रीट्रेंड वजनांसह एक एम्बेडिंग लेयर परिभाषित करू शकतो:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **टीप**: लक्षात घ्या की `Embedding` तयार करताना आम्ही `trainable=False` सेट केले आहे, याचा अर्थ आम्ही Embedding लेयर पुन्हा प्रशिक्षण देत नाही. यामुळे अचूकता थोडी कमी होऊ शकते, परंतु प्रशिक्षणाचा वेग वाढतो.

### एम्बेडिंग शब्दसंग्रह वापरणे

मागील पद्धतीसह एक समस्या म्हणजे TextVectorization आणि Embedding मध्ये वापरलेले शब्दसंग्रह वेगवेगळे आहेत. या समस्येवर मात करण्यासाठी, आपण खालीलपैकी एक उपाय वापरू शकतो:
* आमच्या शब्दसंग्रहावर Word2Vec मॉडेल पुन्हा प्रशिक्षण द्या.
* प्रीट्रेन केलेल्या Word2Vec मॉडेलमधील शब्दसंग्रहासह आमचा डेटासेट लोड करा. डेटासेट लोड करताना वापरलेले शब्दसंग्रह निर्दिष्ट केले जाऊ शकतात.

दुसरी पद्धत सोपी वाटते, म्हणून ती अंमलात आणूया. सर्वप्रथम, आम्ही Word2Vec एम्बेडिंगमधून घेतलेल्या निर्दिष्ट शब्दसंग्रहासह `TextVectorization` लेयर तयार करू:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

जेनसिम वर्ड एम्बेडिंग्ज लायब्ररीमध्ये एक सोयीस्कर फंक्शन, `get_keras_embeddings`, आहे, जे तुमच्यासाठी स्वयंचलितपणे संबंधित केरस एम्बेडिंग्ज लेयर तयार करेल.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


आम्ही उच्च अचूकता पाहत नाही याचे एक कारण म्हणजे आमच्या डेटासेटमधील काही शब्द प्रीट्रेन केलेल्या GloVe शब्दसंग्रहात नसल्यामुळे ते मूलत: दुर्लक्षित केले जातात. यावर मात करण्यासाठी, आम्ही आमच्या डेटासेटच्या आधारे स्वतःचे एम्बेडिंग्स प्रशिक्षण देऊ शकतो.


## संदर्भात्मक एम्बेडिंग्स

Word2Vec सारख्या पारंपरिक प्रीट्रेन केलेल्या एम्बेडिंग्सचे एक मुख्य मर्यादित वैशिष्ट्य म्हणजे, जरी त्या एखाद्या शब्दाचा काही अर्थ पकडू शकतात, तरी त्या वेगवेगळ्या अर्थांमध्ये फरक करू शकत नाहीत. यामुळे डाउनस्ट्रीम मॉडेल्समध्ये समस्या निर्माण होऊ शकतात.

उदाहरणार्थ, 'play' या शब्दाचा दोन वेगवेगळ्या वाक्यांमध्ये वेगळा अर्थ आहे:
- मी थिएटरमध्ये एक **play** पाहायला गेलो.
- जॉनला त्याच्या मित्रांसोबत **play** करायचे आहे.

आपण ज्या प्रीट्रेन केलेल्या एम्बेडिंग्सबद्दल बोललो, त्या 'play' या शब्दाच्या दोन्ही अर्थांना एकाच एम्बेडिंगमध्ये दर्शवतात. ही मर्यादा दूर करण्यासाठी, आपल्याला **भाषा मॉडेल**वर आधारित एम्बेडिंग्स तयार करणे आवश्यक आहे, जे मोठ्या प्रमाणावर मजकूरावर प्रशिक्षित केले जाते आणि *जाणते* की शब्द वेगवेगळ्या संदर्भांमध्ये कसे एकत्र ठेवले जाऊ शकतात. संदर्भात्मक एम्बेडिंग्सवर चर्चा करणे या ट्यूटोरियलच्या कक्षेबाहेर आहे, परंतु आपण त्यांच्याबद्दल पुढील युनिटमध्ये भाषा मॉडेल्सवर चर्चा करताना परत येऊ.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
